## Hello Edge - SDK

In [12]:
import requests
import os
import wallaroo
from wallaroo.object import EntityNotFoundError

In [8]:
import string
import random

# make a random 4 character suffix to verify uniqueness in tutorials
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix='john'

workspace_name = f'edge-publish-demo{suffix}'
pipeline_name = 'edge-pipeline'
model_name = 'ccfraud'
model_file_name = './models/xgboost_ccfraud.onnx'

In [4]:
wl = wallaroo.Client()

Please log into the following URL in a web browser:

	https://keycloak.autoscale-uat-ee.wallaroo.dev/auth/realms/master/device?user_code=RRRB-ZRCZ

Login successful!


In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [6]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'edge-publish-demojohn', 'id': 18, 'archived': False, 'created_by': 'a2a75c12-7cbe-497f-9633-3cd0fcafb33e', 'created_at': '2023-08-21T20:08:12.457471+00:00', 'models': [], 'pipelines': []}

In [9]:
edge_demo_model = wl.upload_model(
    model_name,
    model_file_name,
    framework=wallaroo.framework.Framework.ONNX,
)

In [10]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("900Mi").build()

In [14]:
pipeline = get_pipeline(pipeline_name)
display(pipeline)

pipeline.add_model_step(edge_demo_model)

pub=pipeline.publish(deploy_config)

name,edge-pipeline
created,2023-08-21 20:09:15.846101+00:00
last_updated,2023-08-21 20:09:15.846101+00:00
deployed,(none)
tags,
versions,2d923f22-636b-46ec-a950-989f31a533a5
steps,
published,False


Waiting for pipeline publish... It may take up to 60 sec.
Pipeline is Publishing....Published.


In [15]:
pub

ID,5
Pipeline Version ID,18
Status,Published
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engine:v2023.3.0-main-3707
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:8bb83848-951f-4dff-9bfb-87ae4b2bdaea
Helm Chart URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-pipeline
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:c9d292b31356345b2873bc2a5077fdec57cb0d2de4c490cb0e6b67a27edd5af2
Helm Chart Version,0.0.1-8bb83848-951f-4dff-9bfb-87ae4b2bdaea
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 0.5, 'memory': '900Mi'}, 'requests': {'cpu': 0.5, 'memory': '900Mi'}}}, 'engineAux': {'images': {}}, 'enginelb': {}}"
Created By,a2a75c12-7cbe-497f-9633-3cd0fcafb33e
Created At,2023-08-21 20:09:23.390481+00:00


In [16]:
display(pipeline)

name,edge-pipeline
created,2023-08-21 20:09:15.846101+00:00
last_updated,2023-08-21 20:09:23.336215+00:00
deployed,(none)
tags,
versions,"8bb83848-951f-4dff-9bfb-87ae4b2bdaea, 2d923f22-636b-46ec-a950-989f31a533a5"
steps,
published,True


In [17]:
wl.list_pipelines()

name,created,last_updated,deployed,tags,versions,steps,published
edge-pipeline,2023-21-Aug 20:09:15,2023-21-Aug 20:09:23,(unknown),,"8bb83848-951f-4dff-9bfb-87ae4b2bdaea, 2d923f22-636b-46ec-a950-989f31a533a5",,True


In [18]:
pipeline.publishes()

id,pipeline_version_id,engine_url,pipeline_url,created_by,created_at,updated_at
5,18,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engine:v2023.3.0-main-3707,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:8bb83848-951f-4dff-9bfb-87ae4b2bdaea,a2a75c12-7cbe-497f-9633-3cd0fcafb33e,2023-21-Aug 20:09:23,2023-21-Aug 20:09:23


Running in docker:



Get credentials for docker: type tok= gcloud auth print-access-token 
Deploy docker container: docker run -p 8080:8080 -e DEBUG=true -e OCI_REGISTRY=us-central1-docker.pkg.dev -e CONFIG_CPUS=4 -e OCI_USERNAME=oauth2accesstoken -e OCI_PASSWORD=$tok -e PIPELINE_URL=us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/pipelinecvx86:adef1abe-d141-43b1-8b35-27426a69459f us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engine:v2023.3.0-main-3707

6. Run inference
	curl -X POST localhost:8080/pipelines/pipelinecvx86 -H "Content-Type: application/vnd.apache.arrow.file" --data-binary @/home/jason/projects/edge_demo/image_224x224.arrow

## Hello Edge API

In [19]:
pv=pipeline.versions()[0]
pv.id()

18

In [42]:
wsid = wl.get_current_workspace().id()
data = {"workspace_id": wsid}
headers = {
    "Authorization": wl.auth._bearer_token_str(),
    "Content-Type": "application/json",
}

In [44]:
# # Pipeline Publish

# url = f"{wl.api_endpoint}/v1/api/pipelines/publish"
# resp = requests.post(
#     url,
#     headers=headers,
#     json={"pipeline_id": 1, "pipeline_version_id": 10, "model_config_ids": []},
# )

# assert resp.status_code == 202

<Response [202]>

In [46]:
# resp.text

'{"id":5,"pipeline_version_id":10,"status":"PendingPublish","created_at":"2023-08-17T17:08:51.999168+00:00","updated_at":"2023-08-17T17:08:51.999168+00:00","created_by":"db13ee60-4162-4a42-a571-61c32e225e3e","pipeline_url":null,"engine_url":null,"helm":null,"engine_config":{"engine":{"resources":{"limits":{"cpu":4.0,"memory":"3Gi"},"requests":{"cpu":4.0,"memory":"3Gi"}}},"enginelb":{"resources":{"limits":{"cpu":1.0,"memory":"512Mi"},"requests":{"cpu":0.2,"memory":"512Mi"}}},"engineAux":{}}}'

In [41]:
pv = pipeline.versions()[0]
pv.id()

18

In [43]:
# Publish Status

url = f"{wl.api_endpoint}/v1/api/pipelines/get_publish_status"
resp = requests.post(
    url,
    headers=headers,
    json={"id": pv.id()},
)
# assert resp.status_code == 200

In [44]:
resp.text

'{"msg":"Internal error: Error while converting response Field pipeline_publish_by_pk in response has no value","code":500}'

In [39]:
url = f"{wl.api_endpoint}/v1/api/pipelines/list_publishes_for_pipeline"
resp = requests.post(
    url,
    headers=headers,
    json={"pipeline_id": pipeline.id()},
)
assert resp.status_code == 202

In [40]:
display(resp.text)

'{"pipelines":[{"id":5,"pipeline_version_id":18,"status":"Published","created_at":"2023-08-21T20:09:23.390481+00:00","updated_at":"2023-08-21T20:09:23.390481+00:00","created_by":"a2a75c12-7cbe-497f-9633-3cd0fcafb33e","pipeline_url":"us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:8bb83848-951f-4dff-9bfb-87ae4b2bdaea","engine_url":"us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engine:v2023.3.0-main-3707","helm":{"reference":"us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:c9d292b31356345b2873bc2a5077fdec57cb0d2de4c490cb0e6b67a27edd5af2","chart":"us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-pipeline","version":"0.0.1-8bb83848-951f-4dff-9bfb-87ae4b2bdaea"},"engine_config":{"engine":{"resources":{"limits":{"cpu":0.5,"memory":"900Mi"},"requests":{"cpu":0.5,"memory":"900Mi"}}},"enginelb":{},"engineAux":{"images":{}}}}]}'